In [509]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from sqlalchemy import text
import json
import sys
sys.path.append('C:\\Users\\PC\\Documents\\Matias\\vehiculos_autocosmos')
from db import *

In [510]:
pd.set_option('display.max_columns', None)

In [511]:
df_cars_models_links = pd.DataFrame()
try:
    with engine.connect():
        print("CONNECTED TO POSTGRESQL DATABASE")
        query = text('SELECT * FROM public."Cars_model_links"')
        df = pd.read_sql_query(query, engine)
        df_cars_models_links = pd.concat([df_cars_models_links, df], ignore_index=True)
except Exception as e:
    print(f"Error: {e}")
    print("READING PARQUET DATA")
    df = pd.read_parquet('./CSV/Cars_model_links.parquet')
    df_cars_models_links = pd.concat([df_cars_models_links, df], ignore_index=True)
df_cars_models_links

CONNECTED TO POSTGRESQL DATABASE


,model,price,use_status,url,year,kms,data
0,FIAT Cronos 1.3L Drive Pack Plus CVT,$21.375.000,nuevo,https://www.autocosmos.com.ar/auto/nuevo/fiat/...,None,None,"{""Combustible"": ""nafta"", ""Cilindrada"": ""1332 c..."
1,Peugeot 208 Feline 1.6,$4.500.000,usado,https://www.autocosmos.com.ar/auto/usado/peuge...,2018,36000,"{""Combustible"": ""nafta"", ""Cilindrada"": ""1587 c..."
2,FIAT Cronos 1.3L Drive Pack Plus,$136.000,nuevo,https://www.autocosmos.com.ar/auto/nuevo/fiat/...,None,None,"{""Combustible"": ""nafta"", ""Cilindrada"": ""1332 c..."
3,FIAT Cronos 1.3L Drive Pack Plus,$150.000,nuevo,https://www.autocosmos.com.ar/auto/nuevo/fiat/...,None,None,"{""Combustible"": ""nafta"", ""Cilindrada"": ""1332 c..."
4,FIAT Toro 1.8 Freedom S-Design 4x2 CD Aut,$200.000,nuevo,https://www.autocosmos.com.ar/auto/nuevo/fiat/...,None,None,"{""Combustible"": ""nafta"", ""Cilindrada"": ""1747 c..."
...,...,...,...,...,...,...,...
715,FIAT Cronos 1.3L Drive Pack Plus,$6.000.000,usado,https://www.autocosmos.com.ar/auto/usado/fiat/...,2022,18000,"{""Combustible"": ""nafta"", ""Cilindrada"": ""1332 c..."
716,Volkswagen Amarok DC 4x2 Comfortline,u$s28.000,usado,https://www.autocosmos.com.ar/auto/usado/volks...,2023,35000,"{""Combustible"": ""diesel"", ""Cilindrada"": ""1968 ..."
717,Peugeot 208 Feline 1.6 Tiptronic,$24.500.000,nuevo,https://www.autocosmos.com.ar/auto/nuevo/peuge...,None,None,"{""Combustible"": ""nafta"", ""Cilindrada"": ""1587 c..."
718,FIAT Strada Freedom Cabina Doble 1.4 Firefly,$273.000,nuevo,https://www.autocosmos.com.ar/auto/nuevo/fiat/...,None,None,"{""Combustible"": ""nafta"", ""Cilindrada"": ""1368 c..."


In [512]:
df_cars_models_links['data'] = df_cars_models_links['data'].apply(json.loads)

In [513]:
df_cars_data = pd.DataFrame()

for index, row in df_cars_models_links.iterrows():
  data = row['data']
  model = row['model']
  url = row['url']
  precio = row['price']
  use_status = row['use_status']
  year = row['year']
  kms = row['kms']
  df = pd.json_normalize(data)
  df['model'] = model
  df['price'] = precio
  df['use_status'] = use_status
  df['year'] = year
  df['kms'] = kms
  df_cars_data = pd.concat([df_cars_data, df], ignore_index=True)

In [514]:
df_cars_data.rename(columns={
    'Sistema start / stop': 'Sistema start/stop',
    'Aceleración 0-100 km/h': 'Aceleración (0-100 km/h)',
    'Motor - tracción': 'Motor-tracción',
    'Frenos (del. - tras.)': 'Frenos',
    'Vidrios (del. - tras.)': 'Vidrios',
}, inplace=True)

In [515]:
df_cars_data.head()

,Combustible,Cilindrada,Potencia,Torque,Alimentación,Cilindros,Válvulas,Sistema start/stop,Aceleración (0-100 km/h),Consumo en ciudad,Consumo en ruta,Consumo mixto,Velocidad máxima,Motor-tracción,Transmisión,Neumáticos,Frenos,Suspensión delantera,Suspensión trasera,Largo,Ancho sin espejos,Ancho con espejos,Alto,Distancia entre ejes,Baúl,Tanque de combustible,Peso,Capacidad de carga,Altura de piso,Remolque con frenos,Remolque sin frenos,Aire acondicionado,Alarma de luces encendidas,Asientos delanteros,Asientos traseros,Tapizados,Cierre de puertas,Vidrios,Espejos exteriores,Espejo interior,Faros delanteros,Faros antiniebla,Computadora de a bordo,Dirección asistida,Llantas,Techo solar,Volante,Sensores de estacionamiento,Cámara de visión,ABS,Distribución electrónica de frenado,Airbag,Alarma e inmovilizador de motor,Anclaje para asientos infantiles,Cinturones de seguridad,Otros,Tercera luz de stop,Autobloqueo de puertas con velocidad,Control de estabilidad,Control de tracción,Equipo de música,Parlantes,Conexión auxiliar (iPod y Mp3),Conexión USB,Interfaz bluetooth,Sistema de control por voz para dispositivos,Pantalla,MirrorLink,model,price,use_status,year,kms,Limitador de velocidad,Limpialuneta,Apertura de baúl y tapa de combustible,Sistema de navegación,Capacidad de vadeo,Ángulo de ataque,Ángulo de partida,Ángulo ventral,Capacidad de pendiente,Escalonamiento vertical,Inclinación lateral,Control de velocidad crucero,Palanca de cambios,Asistencia en frenada de emergencia,Reproducción de audio vía bluetooth,Recarga inalámbrica del celular,Encendido del motor,Sistema de estacionamiento asistido,Sensor de lluvia,Apple Carplay / Android Auto,Puertas,Encendido automático de balizas,Wi-Fi,Sistema de video,Potencia total sistema híbrido,Torque total sistema híbrido,Tensión nominal,Tipo,Cantidad de celdas,Cantidad de módulos,Luz de freno variable,T.V.,Neumáticos delanteros,Neumáticos traseros,Preinstalación teléfono
0,nafta,1332 cc,99/6000 cv/rpm,127/4000 N·m/rpm,inyección electrónica multipunto,4 en línea,8,no,"12,8 s",N/D,N/D,N/D,174 km/h,delantero - delantera,automática CVT,185/60/R15,discos sólidos - tambor,independiente McPherson con brazos oscilantes ...,semi-independiente con eje torsional con amort...,4364 mm,1724 mm,1962 mm,1508 mm,2521 mm,525 dm3,48 l,1155 kg,N/D,160 mm,N/D,400 kg,climatizador,sí,con ajuste manual,abatibles completos,tela,centralizado con comando a distancia,eléctricos - eléctricos,eléctricos,antideslumbrante manual,"con apagado automático, fijos",delanteros,sí,eléctrica progresiva,aleación,no tiene,con ajuste en altura,traseros,trasera,sí,sí,conductor y acompañante,alarma e inmovilizador de motor,sí,"delanteros inerciales, traseros inerciales","asistente para arranque en pendientes, indicad...",sí,sí,sí,sí,AM - FM y CD con lector de Mp3,N/D,sí,sí,sí,sí,táctil de 7'',sí,FIAT Cronos 1.3L Drive Pack Plus CVT,$21.375.000,nuevo,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,nafta,1587 cc,115/5800 cv/rpm,152/4000 N·m/rpm,inyección electrónica multipunto,4 en línea,16,no,"11,0 s",N/D,N/D,N/D,195 km/h,delantero - delantera,manual 5 velocidades,195/60/R15,discos ventilados - tambor,independiente tipo McPherson,"traversa deformable, resortes helicoidales, am...",3975 mm,1740 mm,2000 mm,1470 mm,2540 mm,318 dm3,55 l,N/D,N/D,N/D,NaN,NaN,climatizador bizona,sí,"con ajuste en altura solo conductor, con ajust...",abatibles 60/40,tela y cuero,centralizado con comando a distancia,eléctricos - eléctricos,eléctricos,antideslumbrante manual,"con encendido automático, con luces led, fijos",delanteros y traseros,sí,eléctrica progresiva,aleación,panorámico fijo,"con ajuste en altura y profundidad, multifunción",delanteros y traseros,trasera,sí,sí,"conductor y acompañante, de cabeza delanteros ...",alarma e inmovilizador de motor,sí,"delanteros inerciales pirotécnicos, traseros i...",NaN,sí,sí,NaN,NaN,"AM - FM, con lector de Mp3, con 

In [516]:
numeric_columns = ['price', 'year', 'kms', 'Potencia', 'Torque', 'Cilindrada', 'Cilindros', 'Aceleración (0-100 km/h)',
                   'Velocidad máxima', 'Largo', 'Ancho sin espejos', 'Ancho con espejos',
                   'Alto', 'Distancia entre ejes', 'Altura de piso', 'Baúl',
                   'Tanque de combustible', 'Capacidad de carga', 'Peso',
                   'Remolque sin frenos', 'Potencia', 'Torque', 'Consumo en ciudad', 'Consumo mixto', 'Consumo en ruta']

In [517]:
def normalize_price(row):
  price = row['price']
  if price.startswith('$'):
      price = int(price.replace("$", "").replace(".", ""))
  elif price.startswith('u$s'):
      price = int(price.replace("u$s", "").replace(".", "")) * 830
  else:
      print("Moneda no reconocida")
  return price

df_cars_data['price'] = df_cars_data.apply(normalize_price, axis=1)
df_cars_data['Cilindrada'] = df_cars_data['Cilindrada'].str.replace(" cc", "")
df_cars_data['Cilindros'] = df_cars_data['Cilindros'].apply(lambda x: x[0])
df_cars_data['Aceleración (0-100 km/h)'] = df_cars_data['Aceleración (0-100 km/h)'].str.replace(",", ".").str.replace(" s", "")
df_cars_data['Velocidad máxima'] = df_cars_data['Velocidad máxima'].str.replace(" km/h", "")
for col in ['Largo', 'Ancho sin espejos', 'Ancho con espejos', 'Alto', 'Distancia entre ejes', 'Altura de piso']:
    df_cars_data[col] = df_cars_data[col].str.replace(" mm", "")
df_cars_data['Baúl'] = df_cars_data['Baúl'].str.replace(" dm3", "")
df_cars_data['Tanque de combustible'] = df_cars_data['Tanque de combustible'].str.replace(" l", "")
df_cars_data['Capacidad de carga'] = df_cars_data['Capacidad de carga'].str.replace(" kg", "")
df_cars_data['Peso'] = df_cars_data['Peso'].str.replace(" kg", "")
df_cars_data['Remolque sin frenos'] = df_cars_data['Remolque sin frenos'].str.replace(" kg", "")

In [518]:
df_cars_data.head()

,Combustible,Cilindrada,Potencia,Torque,Alimentación,Cilindros,Válvulas,Sistema start/stop,Aceleración (0-100 km/h),Consumo en ciudad,Consumo en ruta,Consumo mixto,Velocidad máxima,Motor-tracción,Transmisión,Neumáticos,Frenos,Suspensión delantera,Suspensión trasera,Largo,Ancho sin espejos,Ancho con espejos,Alto,Distancia entre ejes,Baúl,Tanque de combustible,Peso,Capacidad de carga,Altura de piso,Remolque con frenos,Remolque sin frenos,Aire acondicionado,Alarma de luces encendidas,Asientos delanteros,Asientos traseros,Tapizados,Cierre de puertas,Vidrios,Espejos exteriores,Espejo interior,Faros delanteros,Faros antiniebla,Computadora de a bordo,Dirección asistida,Llantas,Techo solar,Volante,Sensores de estacionamiento,Cámara de visión,ABS,Distribución electrónica de frenado,Airbag,Alarma e inmovilizador de motor,Anclaje para asientos infantiles,Cinturones de seguridad,Otros,Tercera luz de stop,Autobloqueo de puertas con velocidad,Control de estabilidad,Control de tracción,Equipo de música,Parlantes,Conexión auxiliar (iPod y Mp3),Conexión USB,Interfaz bluetooth,Sistema de control por voz para dispositivos,Pantalla,MirrorLink,model,price,use_status,year,kms,Limitador de velocidad,Limpialuneta,Apertura de baúl y tapa de combustible,Sistema de navegación,Capacidad de vadeo,Ángulo de ataque,Ángulo de partida,Ángulo ventral,Capacidad de pendiente,Escalonamiento vertical,Inclinación lateral,Control de velocidad crucero,Palanca de cambios,Asistencia en frenada de emergencia,Reproducción de audio vía bluetooth,Recarga inalámbrica del celular,Encendido del motor,Sistema de estacionamiento asistido,Sensor de lluvia,Apple Carplay / Android Auto,Puertas,Encendido automático de balizas,Wi-Fi,Sistema de video,Potencia total sistema híbrido,Torque total sistema híbrido,Tensión nominal,Tipo,Cantidad de celdas,Cantidad de módulos,Luz de freno variable,T.V.,Neumáticos delanteros,Neumáticos traseros,Preinstalación teléfono
0,nafta,1332,99/6000 cv/rpm,127/4000 N·m/rpm,inyección electrónica multipunto,4,8,no,12.8,N/D,N/D,N/D,174,delantero - delantera,automática CVT,185/60/R15,discos sólidos - tambor,independiente McPherson con brazos oscilantes ...,semi-independiente con eje torsional con amort...,4364,1724,1962,1508,2521,525,48,1155,N/D,160,N/D,400,climatizador,sí,con ajuste manual,abatibles completos,tela,centralizado con comando a distancia,eléctricos - eléctricos,eléctricos,antideslumbrante manual,"con apagado automático, fijos",delanteros,sí,eléctrica progresiva,aleación,no tiene,con ajuste en altura,traseros,trasera,sí,sí,conductor y acompañante,alarma e inmovilizador de motor,sí,"delanteros inerciales, traseros inerciales","asistente para arranque en pendientes, indicad...",sí,sí,sí,sí,AM - FM y CD con lector de Mp3,N/D,sí,sí,sí,sí,táctil de 7'',sí,FIAT Cronos 1.3L Drive Pack Plus CVT,21375000,nuevo,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,nafta,1587,115/5800 cv/rpm,152/4000 N·m/rpm,inyección electrónica multipunto,4,16,no,11.0,N/D,N/D,N/D,195,delantero - delantera,manual 5 velocidades,195/60/R15,discos ventilados - tambor,independiente tipo McPherson,"traversa deformable, resortes helicoidales, am...",3975,1740,2000,1470,2540,318,55,N/D,N/D,N/D,NaN,NaN,climatizador bizona,sí,"con ajuste en altura solo conductor, con ajust...",abatibles 60/40,tela y cuero,centralizado con comando a distancia,eléctricos - eléctricos,eléctricos,antideslumbrante manual,"con encendido automático, con luces led, fijos",delanteros y traseros,sí,eléctrica progresiva,aleación,panorámico fijo,"con ajuste en altura y profundidad, multifunción",delanteros y traseros,trasera,sí,sí,"conductor y acompañante, de cabeza delanteros ...",alarma e inmovilizador de motor,sí,"delanteros inerciales pirotécnicos, traseros i...",NaN,sí,sí,NaN,NaN,"AM - FM, con lector de Mp3, con comandos de au...",6,sí,sí,sí,NaN,táctil de 7'',sí,Peugeot 208 Feline 1.6,4500000,usado,2018,360

In [519]:
df_cars_data[numeric_columns]

,price,year,kms,Potencia,Torque,Cilindrada,Cilindros,Aceleración (0-100 km/h),Velocidad máxima,Largo,Ancho sin espejos,Ancho con espejos,Alto,Distancia entre ejes,Altura de piso,Baúl,Tanque de combustible,Capacidad de carga,Peso,Remolque sin frenos,Potencia,Torque,Consumo en ciudad,Consumo mixto,Consumo en ruta
0,21375000,None,None,99/6000 cv/rpm,127/4000 N·m/rpm,1332,4,12.8,174,4364,1724,1962,1508,2521,160,525,48,N/D,1155,400,99/6000 cv/rpm,127/4000 N·m/rpm,N/D,N/D,N/D
1,4500000,2018,36000,115/5800 cv/rpm,152/4000 N·m/rpm,1587,4,11.0,195,3975,1740,2000,1470,2540,N/D,318,55,N/D,N/D,NaN,115/5800 cv/rpm,152/4000 N·m/rpm,N/D,N/D,N/D
2,136000,None,None,99/6000 cv/rpm,127/4000 N·m/rpm,1332,4,11.5,174,4364,1724,1962,1508,2521,160,525,48,N/D,1155,400,99/6000 cv/rpm,127/4000 N·m/rpm,N/D,N/D,N/D
3,150000,None,None,99/6000 cv/rpm,127/4000 N·m/rpm,1332,4,11.5,174,4364,1724,1962,1508,2521,160,525,48,N/D,1155,400,99/6000 cv/rpm,127/4000 N·m/rpm,N/D,N/D,N/D
4,200000,None,None,130/6000 cv/rpm,171/3750 N·m/rpm,1747,4,14,176,4945,1845,2033,1735,2990,211-264,937,60,650,1688,400,130/6000 cv/rpm,171/3750 N·m/rpm,N/D,N/D,N/D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,6000000,2022,18000,99/6000 cv/rpm,127/4000 N·m/rpm,1332,4,11.5,174,4364,1724,1962,1508,2521,160,525,48,N/D,1155,400,99/6000 cv/rpm,127/4000 N·m/rpm,N/D,N/D,N/D
716,23240000,2023,35000,180/4000 cv/rpm,400/1500-2250 N·m/rpm,1968,4,10.3,184,5254,1944,1954,1834,3097,226,N/A,80,1119,1921,750,180/4000 cv/rpm,400/1500-2250 N·m/rpm,N/D,"7,9 l/100km",N/D
717,24500000,None,None,115/6000 cv/rpm,150/4000 N·m/rpm,1587,3,N/D,190,4055,1738,1960,1453,2538,N/D,311,47,N/D,1111,NaN,115/6000 cv/rpm,150/4000 N·m/rpm,N/D,N/D,N/D
718,273000,None,None,85/5750 cv/rpm,122/3500 N·m/rpm,1368,4,14.7,154,4474,1781,1967,N/D,2737,191,683,55,650,1161,400,85/5750 cv/rpm,122/3500 N·m/rpm,N/D,N/D,N/D


In [520]:
df_cars_data[['potencia_cv', 'potencia_rpm']] = df_cars_data['Potencia'].str.replace(" cv/rpm", "").str.split("/", expand=True)
df_cars_data[['torque_nm', 'torque_rpm']] = df_cars_data['Torque'].str.replace(" N·m/rpm", "").str.split("/", expand=True)

df_cars_data[['potencia_cv', 'potencia_rpm', 'torque_nm', 'torque_rpm']] = df_cars_data[['potencia_cv', 'potencia_rpm', 'torque_nm', 'torque_rpm']]\
.replace(r'(\d+)-(\d+)', r'\1', regex=True)\
.replace(r'(\d+) - (\d+)', r'\1', regex=True)\
.replace('', '1')\
.replace('N', '1')\
.replace('D', '1')\
.fillna(1)\
.replace('0', '1')\
.replace('128 cv', '128')\
.replace(r'\..*', '', regex=True)

def calculate_power_and_torque(row):
  power_cv = row['potencia_cv']
  power_rpm = row['potencia_rpm']
  torque_nm = row['torque_nm']
  torque_rpm = row['torque_rpm'] 
  power_calculated = (int(power_cv) * 735.49875) / int(power_rpm)
  torque_calculated = int(torque_nm) / int(torque_rpm)
  # print({'power_calculated': round(power_calculated, 2)})
  # print({'torque_calculated': round(torque_calculated, 2)})
  return round(power_calculated, 2), round(torque_calculated, 2)

df_cars_data['Potencia'], df_cars_data['Torque'] = zip(*df_cars_data.apply(calculate_power_and_torque, axis=1))

C:\Users\PC\AppData\Local\Temp\ipykernel_8560\2991794825.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_cars_data[['potencia_cv', 'potencia_rpm']] = df_cars_data['Potencia'].str.replace(" cv/rpm", "").str.split("/", expand=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_8560\2991794825.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_cars_data[['potencia_cv', 'potencia_rpm']] = df_cars_data['Potencia'].str.replace(" cv/rpm", "").str.split("/", expand=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_8560\2991794825.py:

In [521]:
print("--------------------------------Consumo en ciudad---------------------------------------")
print(df_cars_data['Consumo en ciudad'].unique())
print("--------------------------------Consumo en ruta---------------------------------------")
print(df_cars_data['Consumo en ruta'].unique())
print("--------------------------------Consumo mixto---------------------------------------")
print(df_cars_data['Consumo mixto'].unique())

df_cars_data[['Consumo en ciudad', 'Consumo en ruta', 'Consumo mixto']] = df_cars_data[['Consumo en ciudad', 'Consumo en ruta', 'Consumo mixto']].replace(['N/D', float('nan')], '0 l/100km')

def calculate_consume(row):
    city_consume = row['Consumo en ciudad'].split(" ")[0].replace(",", ".") if row['Consumo en ciudad'] != 'N/D' or row['Consumo en ciudad'] != None else 'N/D'
    route_consume = row['Consumo en ruta'].split(" ")[0].replace(",", ".") if row['Consumo en ruta'] != 'N/D' or row['Consumo en ruta'] != None else 'N/D'
    mix_consume = row['Consumo mixto'].split(" ")[0].replace(",", ".") if row['Consumo mixto'] != 'N/D' or row['Consumo mixto'] != None else 'N/D'
    return pd.Series([city_consume, route_consume, mix_consume])

df_cars_data[['Consumo en ciudad', 'Consumo en ruta', 'Consumo mixto']] = df_cars_data.apply(calculate_consume, axis=1)

print("--------------------------------Consumo en ciudad---------------------------------------")
print(df_cars_data['Consumo en ciudad'].unique())
print("--------------------------------Consumo en ruta---------------------------------------")
print(df_cars_data['Consumo en ruta'].unique())
print("--------------------------------Consumo mixto---------------------------------------")
print(df_cars_data['Consumo mixto'].unique())

--------------------------------Consumo en ciudad---------------------------------------
['N/D' nan '9,3 l/100km' '9,7 l/100km' '10,0 l/100km' '10,4 l/100km'
 '12,0 l/100km' '' '8,9 l/100km' '9,9 l/100km' '6,3 l/100km'
 '11,0 l/100km' '8,7 l/100km' '7,0 l/100km' '12,9 l/100km' '8,5 l/100km'
 '00 l/100km' '10,6 l/100km' '5,9 l/100km' '6,1 l/100km' '15,0 l/100km'
 '12,2 l/100km' '10,1 l/100km' '5,5 l/100km' '7,4 l/100km' '8.6 l/100km'
 '9,1 l/100km' '7.8 l/100km' '9.9 l/100km' '6,4 l/100km' '12,3 l/100km']
--------------------------------Consumo en ruta---------------------------------------
['N/D' nan '5,3 l/100km' '7,9 l/100km' '5,8 l/100km' '6,0 l/100km'
 '6,9 l/100km' '4,9 l/100km' '5,6 l/100km' '4,0 l/100km' '6,2 l/100km'
 '5,7 l/100km' '4,7 l/100km' '8,6 l/100km' '5,9 l/100km' '8,0 l/100km'
 '6,5 l/100km' '4,3 l/100km' '5,5 l/100km' '9,3 l/100km' '7,1 l/100km'
 '15,2 l/100km' '3,7 l/100km' '6,6 l/100km' '5,2 l/100km' '5,4 l/100km'
 '4,8 l/100km' '5,0 l/100km' '4,5 l/100km' '8,9 l/1

In [522]:
for col in numeric_columns:
    df_cars_data[col] = pd.to_numeric(df_cars_data[col], downcast='signed', errors='coerce')

In [523]:
df_cars_data[numeric_columns]

,price,year,kms,Potencia,Torque,Cilindrada,Cilindros,Aceleración (0-100 km/h),Velocidad máxima,Largo,Ancho sin espejos,Ancho con espejos,Alto,Distancia entre ejes,Altura de piso,Baúl,Tanque de combustible,Capacidad de carga,Peso,Remolque sin frenos,Potencia,Torque,Consumo en ciudad,Consumo mixto,Consumo en ruta
0,21375000,NaN,NaN,12.14,0.03,1332.0,4.0,12.8,174.0,4364.0,1724.0,1962.0,1508.0,2521.0,160.0,525.0,48.0,NaN,1155.0,400.0,12.14,0.03,0.0,0.0,0.0
1,4500000,2018.0,36000.0,14.58,0.04,1587.0,4.0,11.0,195.0,3975.0,1740.0,2000.0,1470.0,2540.0,NaN,318.0,55.0,NaN,NaN,NaN,14.58,0.04,0.0,0.0,0.0
2,136000,NaN,NaN,12.14,0.03,1332.0,4.0,11.5,174.0,4364.0,1724.0,1962.0,1508.0,2521.0,160.0,525.0,48.0,NaN,1155.0,400.0,12.14,0.03,0.0,0.0,0.0
3,150000,NaN,NaN,12.14,0.03,1332.0,4.0,11.5,174.0,4364.0,1724.0,1962.0,1508.0,2521.0,160.0,525.0,48.0,NaN,1155.0,400.0,12.14,0.03,0.0,0.0,0.0
4,200000,NaN,NaN,15.94,0.05,1747.0,4.0,14.0,176.0,4945.0,1845.0,2033.0,1735.0,2990.0,NaN,937.0,60.0,650.0,1688.0,400.0,15.94,0.05,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,6000000,2022.0,18000.0,12.14,0.03,1332.0,4.0,11.5,174.0,4364.0,1724.0,1962.0,1508.0,2521.0,160.0,525.0,48.0,NaN,1155.0,400.0,12.14,0.03,0.0,0.0,0.0
716,23240000,2023.0,35000.0,33.10,0.27,1968.0,4.0,10.3,184.0,5254.0,1944.0,1954.0,1834.0,3097.0,226.0,NaN,80.0,1119.0,1921.0,750.0,33.10,0.27,0.0,7.9,0.0
717,24500000,NaN,NaN,14.10,0.04,1587.0,3.0,NaN,190.0,4055.0,1738.0,1960.0,1453.0,2538.0,NaN,311.0,47.0,NaN,1111.0,NaN,14.10,0.04,0.0,0.0,0.0
718,273000,NaN,NaN,10.87,0.03,1368.0,4.0,14.7,154.0,4474.0,1781.0,1967.0,NaN,2737.0,191.0,683.0,55.0,650.0,1161.0,400.0,10.87,0.03,0.0,0.0,0.0


In [524]:
df_cars_data.sample(15)

,Combustible,Cilindrada,Potencia,Torque,Alimentación,Cilindros,Válvulas,Sistema start/stop,Aceleración (0-100 km/h),Consumo en ciudad,Consumo en ruta,Consumo mixto,Velocidad máxima,Motor-tracción,Transmisión,Neumáticos,Frenos,Suspensión delantera,Suspensión trasera,Largo,Ancho sin espejos,Ancho con espejos,Alto,Distancia entre ejes,Baúl,Tanque de combustible,Peso,Capacidad de carga,Altura de piso,Remolque con frenos,Remolque sin frenos,Aire acondicionado,Alarma de luces encendidas,Asientos delanteros,Asientos traseros,Tapizados,Cierre de puertas,Vidrios,Espejos exteriores,Espejo interior,Faros delanteros,Faros antiniebla,Computadora de a bordo,Dirección asistida,Llantas,Techo solar,Volante,Sensores de estacionamiento,Cámara de visión,ABS,Distribución electrónica de frenado,Airbag,Alarma e inmovilizador de motor,Anclaje para asientos infantiles,Cinturones de seguridad,Otros,Tercera luz de stop,Autobloqueo de puertas con velocidad,Control de estabilidad,Control de tracción,Equipo de música,Parlantes,Conexión auxiliar (iPod y Mp3),Conexión USB,Interfaz bluetooth,Sistema de control por voz para dispositivos,Pantalla,MirrorLink,model,price,use_status,year,kms,Limitador de velocidad,Limpialuneta,Apertura de baúl y tapa de combustible,Sistema de navegación,Capacidad de vadeo,Ángulo de ataque,Ángulo de partida,Ángulo ventral,Capacidad de pendiente,Escalonamiento vertical,Inclinación lateral,Control de velocidad crucero,Palanca de cambios,Asistencia en frenada de emergencia,Reproducción de audio vía bluetooth,Recarga inalámbrica del celular,Encendido del motor,Sistema de estacionamiento asistido,Sensor de lluvia,Apple Carplay / Android Auto,Puertas,Encendido automático de balizas,Wi-Fi,Sistema de video,Potencia total sistema híbrido,Torque total sistema híbrido,Tensión nominal,Tipo,Cantidad de celdas,Cantidad de módulos,Luz de freno variable,T.V.,Neumáticos delanteros,Neumáticos traseros,Preinstalación teléfono,potencia_cv,potencia_rpm,torque_nm,torque_rpm
287,nafta,1368.0,10.87,0.03,inyección electrónica multipunto,4.0,8,N/D,14.7,0.0,0.0,0.0,154.0,delantero - delantera,manual 5 velocidades,195/65/R15,discos ventilados - tambor,independiente tipo McPherson,eje rígido,4474.0,1781.0,1967.0,NaN,2737.0,683.0,55.0,1161.0,650.0,191.0,N/D,400.0,manual,sí,"con ajuste en altura solo conductor, con ajust...",N/D,tela,centralizado,eléctricos - eléctricos,"con luz de giro incorporadas, eléctricos",antideslumbrante manual,fijos,delanteros,sí,hidráulica,aleación,no tiene,"con ajuste en altura, multifunción",NaN,NaN,sí,sí,"conductor y acompañante, laterales delanteros",alarma,NaN,"delanteros inerciales, traseros inerciales","asistente para arranque en pendientes, indicad...",sí,sí,sí,sí,"AM - FM, con lector de Mp3, con comandos de au...",N/D,sí,NaN,sí,NaN,"color de 3,5""",NaN,FIAT Strada Freedom Cabina Doble 1.4 Firefly,21000000,nuevo,NaN,NaN,NaN,NaN,interna y a distancia solo tapa de combustible,NaN,NaN,"23,8°","26,6°",N/D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85,5750,122,3500
471,nafta,1399.0,22.51,0.12,inyección directa turbo,4.0,16,sí,8.5,0.0,0.0,0.0,220.0,delantero - delantera,automática 6 velocidades,215/50/R17,discos ventilados - discos sólidos,independiente tipo McPherson,semi-independiente con eje torsional con amort...,4448.0,1807.0,2042.0,1484.0,2700.0,290.0,52.0,NaN,NaN,NaN,NaN,NaN,climatizador,sí,con ajuste manual,abatibles 60/40,cuero,centralizado con comando a distancia,eléctricos - eléctricos,"con luz de giro incorporadas, eléctricos",antideslumbrante manual,con regulación interna,traseros,sí,eléctrica progresiva,aleación,no tiene,"con ajuste en altura y profundidad, multifunci...",traseros,trasera,sí,sí,"conductor y acompañante, de cabeza delanteros ...",alarma e inmovilizador de motor,sí,"delanteros inerciales, traseros inerciales (2)...",indicador de presión de neumáticos,sí,sí,sí,sí,"AM - FM y CD con lector de Mp3, con comandos d...",6,sí,sí,sí,NaN,táctil de 7''

In [525]:
df_cars_data.to_sql(f'Cars_model_data', engine, if_exists='replace', schema='public', index=False)
print(F"TABLE Cars_model_data WAS SAVED SUCCESSFULY")
df_cars_data.to_csv('../CSV/Cars_model_data.parquet')

TABLE Cars_model_data WAS SAVED SUCCESSFULY
